## 1. Import all the packages/libraries first! <font color="#FF0000">(Execute Only)</font>

In [ ]:
#=========== RUN ME =========== 
import os
import re
import ast
import glob
import asyncio
import pandas as pd
import dask.dataframe as dd
import numpy as np
import swifter
import functools
import ipywidgets as widgets
# from yaspin import yaspin
from os import path
from os import listdir
from os import remove
# from threading import Timer
from collections import Counter
from collections import OrderedDict
from datetime import datetime
from tqdm import tqdm
# from rich.progress import track
# from alive_progress import alive_bar
from modin.config import ProgressBar
ProgressBar.enable()
from IPython.display import display, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual, \
HBox, VBox, Button, HTML, Tab, Output
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

### 1.1 Set Global Variables for your project <font color="#FF0000">(Execute Only)</font>

In [ ]:
#Get absolute path instead of relative path
DIR = path.dirname(path.abspath("__file__"))
CSV_PATH = None
CSV_PATH2 = None
DEST_PATH = None
CFIDF_CSV_PATH = None
TFIDF_CSV_PATH = None
RANGE = 0

### 1.2 Initialize Widgets #Debounce & Main <font color="#FF0000">(Execute Only)</font>

In [ ]:
# Debounce
class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator
    
# Folder Dropdown
foldersOnly = list(filter(lambda x: path.isdir(x), listdir(DIR)))

def Dropdown(options, desc):
  return widgets.Dropdown(
    options=options,
    description=desc,
    disabled=False,
  )

# Folder Path Display Dropdown
folderPathLayout = widgets.Layout(width='auto')

def folderDisplay(desc):
  return widgets.Text(
    value='',
    placeholder='',
    description=desc,
    disabled=True,
    layout=folderPathLayout
)

# Button
def button(desc):
  return widgets.Button(description=desc)

## Bounded Integer
def boundedInt():
  return widgets.BoundedIntText(
    value=0,
    min=1,
    step=1,
    description=f'Position: ',
    disabled=False
  )

## Text
def inputText(desc):
  return widgets.Text(
    value='',
    placeholder='Type something here',
    description=desc,
    disabled=False
)

### 1.3 Initialize Widgets #Functions <font color="#FF0000">(Execute Only)</font>

In [2]:
foldersOnly = list(filter(lambda x: path.isdir(x), listdir(DIR)))
#Widgets
# ===== Folder =====
folder_dropdown1 = Dropdown(foldersOnly, 'Folders: ')
# ===== File =====
file_dropdown1 = Dropdown([], 'Select Folder: ')

file_dropdown1.style = {'description_width': 'initial'}
# ===== Select Column =====
columnsDropdown_1 = Dropdown([], 'Select Column: ')
columnsDropdown_2 = Dropdown([], 'Select Column: ')
columnsDropdown_3 = Dropdown([], 'Select Column: ')

columnsDropdown_1.style = {'description_width': 'initial'}
columnsDropdown_2.style = {'description_width': 'initial'}
columnsDropdown_3.style = {'description_width': 'initial'}
# ==== Input Text ====
input_Text = inputText('File name:')
# ===== Folder path display =====
folder_pathDisplay1 = folderDisplay('Folder Path: ')
# =====
btn_load1 = button('Load Source!')
btn_load2 = button('Load Previous!')
btn_load3 = button('Load Previous!')
btn_setDest = button('Set Path!')
btn_setRange = button('Set Range!')
btn_setRange.style = {'description_width': 'initial'}
btn_exeFx = button('Execute!')
btn_setFileName = button('Set!')

btn_setRange.style = {'description_width': 'initial'}
# =====
input_boundedInt = boundedInt()
# =====

# ===========================
# ===========================
# ===========================

# Functions
@debounce(0.5)
def changeFolder(change):
  folder_pathDisplay1.value = path.join(DIR, change['new'])
  file_dropdown1.options = listdir(folder_pathDisplay1.value)

# Functions for Reading Corpus
def loadDataset(btn):
  global CSV_PATH
  CSV_PATH = glob.glob(path.join(folder_pathDisplay1.value, file_dropdown1.value, '*.csv'))
  print(f"Successfully set {file_dropdown1.value} as dataset source")

def loadDataset_clear(btn):
  global CSV_PATH
  CSV_PATH = glob.glob(path.join(folder_pathDisplay1.value, file_dropdown1.value, '*.csv'))
  # clear_output(wait=True)
  print(f"Successfully set {file_dropdown1.value} as dataset source")

def loadDataset_clear2(btn):
  global CSV_PATH2
  CSV_PATH2 = glob.glob(path.join(folder_pathDisplay1.value, file_dropdown1.value, '*.csv'))
  # clear_output(wait=True)
  print(f"Successfully set {file_dropdown1.value} as dataset source")

def loadTFIDFDataset(btn):
  global TFIDF_CSV_PATH
  TFIDF_CSV_PATH = glob.iglob(path.join(folder_pathDisplay1.value, file_dropdown1.value, '*.csv'))
  print(f"Successfully set {TFIDF_CSV_PATH} as TFIDF dataset source")

def loadCFIDFDataset(btn):
  global CFIDF_CSV_PATH
  CFIDF_CSV_PATH = glob.iglob(path.join(folder_pathDisplay1.value, file_dropdown1.value, '*.csv'))
  print(f"Successfully set {CFIDF_CSV_PATH} as CSIDF dataset source")

def setDestination(btn):
  global RANGE, DEST_PATH
  RANGE = 1
  DEST_PATH = path.join(folder_pathDisplay1.value, file_dropdown1.value)
  print(f"Successfully set {DEST_PATH} as destination")
  print(f"Default file starting position is {RANGE}")

def setRange(btn):
  global RANGE
  RANGE = input_boundedInt.value
  print(f"Successfully set file starting position to {RANGE}")
  
folder_dropdown1.observe(changeFolder, 'value')

NameError: name 'listdir' is not defined

### 1.4 Initialize Widgets #Tabs <font color="#FF0000">(Execute Only)</font>

In [ ]:
tabTitles = ["Load Dataset", "Load Previous Output", "Load Previous Output", "Set Output Destination", "Set Range", "Execute!"]
tabTitles2 = ["Load Dataset", "Set File Name", "Select Column", "Load Previous Output", "Load Previous Output", "Set Output Destination", "Set Range", "Execute!"]
#Tab
tab = Tab()
#Tab Widgets
tab_load1 = VBox([
  widgets.HTML(value="<b><font style='color:red'>Select & Load your dataset Folder:</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load1
])

tab_setFileName = VBox([
  widgets.HTML(value="<b>Default output to main folder not assets!!</b>"),
  widgets.HTML(value="<b><font style='color:red'>Set output filename:</font></b>"),
  input_Text,
  btn_setFileName
])

tab_loadPrevious = VBox([
  widgets.HTML(value="Select & Load your previous output folder"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load2
])

tab_loadPrevious2 = VBox([
  widgets.HTML(value="Select & Load your previous output folder"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load3
])

tab_setDest = VBox([
  widgets.HTML(value="Select Output Destination: <b><font style='color:red'>Make sure you have created a folder first!</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_setDest
  ])

tab_setRange = VBox([
  widgets.HTML(value="Set the Starting Position for processing: <b>Default processing the <font style='color:red'>FIRST</font> file</b>"),
  input_boundedInt,
  btn_setRange
])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])

## 3. All related functions Initialization (Path need modification)

### 3.1 Initialize Jieba <font color="#FF0000">(Execute Only)</font>

In [ ]:
#Jieba Initialization
import jieba_pyfast2 as jbf2
jbf2.set_dictionary('./assets/dict_big.txt')
jbf2.load_userdict('./assets/jieba_canton_dict.txt')

### 3.2 Reading Concept Corpus <font color="#FF0000">(Execute Only)</font>

In [ ]:
conceptDict = {}
PATH = glob.iglob(path.join(DIR, 'assets/concept_corpus/*.csv'))
# Functions for Reading Corpus
## Pre-processing
def trimTerms(content):
  return str(content).strip()

def loadConceptCorpus():
  global conceptDict
  for CSV in PATH:
    name = 'cfidf_concept_' + path.basename(CSV).split('.')[0]
    conceptDict[name] = pd.read_csv(CSV, encoding='utf-8-sig', index_col=False)['Term'].to_numpy()
  for key, value in conceptDict.items():
    conceptDict[key] = np.vectorize(trimTerms)(value)
  print("Done!")

loadConceptCorpus()

### 3.3 Reading Stop Word <font color="#FF0000">(Execute Only)</font>

In [ ]:
df_stopword = None
PATH = path.join(DIR, 'assets/stopwords.txt')
# Functions for Reading Stop Word
def loadStopWord():
  global df_stopword
  df_stopword = pd.read_csv(PATH, delimiter="\s+", header=None, names=['word'])['word']
  df_stopword = list(df_stopword)
  df_stopword.append(' ')
  print("Done!")

loadStopWord()

### 3.4 Segmentation functions & Terms Counting functions (TFIDF) <font color="#FF0000">(Execute Only)</font>

In [ ]:
# === RUN ===
#Function for segmentating strings (Normal)
def segmentation(row):
    return list(filter(lambda x: x not in df_stopword, jbf2.lcut(str(row), HMM=True)))

# Function for segnentating strings (Tfidf)
# To fit the pattern required by scikit learn
# Also filter out segmented words/terms that defined in "Stop Word Corpus"
# Pattern Reference: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer
def segment_df_tfidf(row):
    content = ' '.join(segmentation(row))
    return content

def segmented_df_tfidf(row):
    content = ' '.join(ast.literal_eval(row))
    return content

def countTerms(content):
  return dict(Counter(filter(lambda x: x not in df_stopword, content)))
  
def countTermsByCat(counter):
  columns = {}
  columns['total_no_of_terms'] = sum(counter.values())
  for key, value in conceptDict.items():
    columns[key] = sum(map(lambda x: counter.get(x), filter(lambda x: x in value, counter.keys())))
  return pd.Series(columns)
  

### 3.5 Pre-processing functions for CFIDF <font color="#FF0000">(Execute Only)</font>

In [1]:
# === RUN ===
#Function for segmentating strings (Normal)
def segmentation(row):
    return list(filter(lambda x: x not in df_stopword, jbf2.lcut(str(row), HMM=True)))

def segmented_replaceTerms_cfidf(row):
  # terms = segmentation(content)
  processedTerms = replaceTermsByConcept(ast.literal_eval(row))
  return processedTerms

def countTerms(content):
  return dict(Counter(filter(lambda x: x not in df_stopword, content)))

def segment_df_cfidf(row):
    terms = segmentation(row)
    processedTerms = replaceTermsByConcept(terms)
    content = ' '.join(processedTerms)
    return content

def join_segmented_df_cfidf(row):
    # processedTerms = replaceTermsByConcept(ast.literal_eval(row))
    content = ' '.join(ast.literal_eval(row))
    return content

def segmented_df_cfidf_list(row):
    processedTerms = replaceTermsByConcept(ast.literal_eval(row))
    return processedTerms

# Match terms by concept for Counter
def matchTermsByConcept_counter(counter):
  keys = counter.keys()
  tempDict = dict()
  def checkAndcreate(term, concept):
    if term in conceptDict[concept]:
      if tempDict.get(concept, None) == None:
        tempDict[concept] = [term]
      else:
        tempDict[concept] += [term]
  
  for concept in conceptDict.keys():
    for term in keys:
      checkAndcreate(term, concept)
  
  return tempDict

# Match terms by concept for list
def matchTermsByConcept_list(terms):
  tempDict = dict()
  
  def checkAndcreate(term, concept):
    if term in conceptDict[concept]:
      if tempDict.get(concept, None) == None:
        tempDict[concept] = [term]
      else:
        tempDict[concept] += [term]
  
  for concept in conceptDict.keys():
    for term in terms:
      checkAndcreate(term, concept)
  
  return tempDict

# Replace terms by concept
def replaceTermsByConcept(terms):
  #remove duplicate
  def removeDuplicate(segmentedTerms):
    return list(OrderedDict.fromkeys(segmentedTerms))

  termsClean = removeDuplicate(terms)
  matchedTermsDict = matchTermsByConcept_list(termsClean) #Pass cleaned terms to find matched terms
  # print(matchedTermsDict)
  result = terms #get the original terms
  for key in matchedTermsDict.keys():
    for term in matchedTermsDict[key]:
      result = [x.replace(term, key) if x == term else x for x in result] #Replace matched terms with concept
  return result

def cfidfTermsCounting(counter):
  columns = {}
  columns['total_no_of_terms'] = sum(counter.values())
  concepts = conceptDict.keys()
  columns['total_no_of_cfidf_concept_terms'] = sum([counter.get(corpus, 0) for corpus in concepts])
  for key, value in conceptDict.items():
    name = 'no_of_' + key + '_terms'
    columns[name] = counter.get(key, 0)
  return pd.Series(columns)

def cfidfCommentsCounting(content):
  columns = {}
  columns['total_no_of_comments'] = 1
  for key, value in conceptDict.items():
    name = 'no_of_' + key + '_comments'
    columns[name] = 1 if key in content else 0
  return pd.Series(columns)